In [5]:
# importing the libraries
import pandas as pd

In [7]:
# Reading the fortune1000 data into a Pandas dataframe
original_fortune_data = pd.read_csv("../DataSets/fortune1000-final.csv", encoding = "ISO-8859-1")

In [11]:
# Finding the size of the data
original_fortune_data.shape

(1000, 19)

In [37]:
# Viewing records
#extracting top 500 rank companies
original_fortune_data = original_fortune_data[original_fortune_data["rank"] < 501]
original_fortune_data.head()

,rank,title,Previous Rank,Revenues ($M),Revenue Change,Profits ($M),Profit Change,Assets ($M),Mkt Value as of 3/29/18 ($M),Employees,CEO,CEO Title,Sector,Industry,Years on Fortune 500 List,City,State,Latitude,Longitude
96,97,3M,94,"$31,657",5.10%,"$4,858.00",-3.80%,"$37,987","$130,550","91,536",Inge G. Thulin,"Chairman, President & Chief Executive Officer",Industrials,Miscellaneous,24,St Paul,MN,44.953703,-93.089958
403,404,A-Mark Precious Metals,395,"$6,990",3.00%,$7.10,-23.60%,$479,$86,126,Gregory N. Roberts,Chairman & Chief Executive Officer,Materials,Miscellaneous,4,El Segundo,CA,33.919180,-118.416465
497,498,ABM Industries,500,"$5,454",6.00%,$3.80,-93.40%,"$3,813","$2,200","140,000",Scott B. Salmirs,"President, Chief Executive Officer & Director",Business Services,Diversified Outsourcing Services,3,New York,NY,40.712775,-74.005973
242,243,ADP,240,"$12,380",6.10%,"$1,733.40",16.10%,"$37,180","$50,302","58,000",Carlos A. Rodriguez,"President, Chief Executive Officer & Director",Business Services,Diversified Outsourcing Services,24,Roseland,NJ,40.820656,-74.293759
163,164,AECOM,161,"$18,203",4.60%,$339.40,253.10%,"$14,397","$5,671","87,000",Michael S. Burke,Chairman & Chief Executive Officer,Engineering & Construction,"Engineering, Construction",10,Los Angeles,CA,34.052234,-118.243685


In [9]:
original_fortune_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 19 columns):
rank                            1000 non-null int64
title                           1000 non-null object
Previous Rank                   938 non-null object
Revenues ($M)                   1000 non-null object
Revenue Change                  1000 non-null object
Profits ($M)                    1000 non-null object
Profit Change                   1000 non-null object
Assets ($M)                     1000 non-null object
Mkt Value as of 3/29/18 ($M)    1000 non-null object
Employees                       1000 non-null object
CEO                             1000 non-null object
CEO Title                       1000 non-null object
Sector                          1000 non-null object
Industry                        1000 non-null object
Years on Fortune 500 List       1000 non-null object
City                            999 non-null object
State                           1000 non-null obj

In [35]:
# reading the S & P data
s_and_p_data = pd.read_csv("../DataSets/constituents_csv.csv", encoding = "ISO-8859-1")
s_and_p_data = s_and_p_data[["Symbol", "Name"]]
s_and_p_data.columns = ['stock', 'title']
s_and_p_data.shape

(505, 2)

In [26]:
s_and_p_data.head()

,stock,title
0,MMM,3M Company
1,AOS,A.O. Smith Corp
2,ABT,Abbott Laboratories
3,ABBV,AbbVie Inc.
4,ACN,Accenture plc


In [38]:
# Sorting both datasets and merging
original_fortune_data = original_fortune_data.sort_values(by=['title'])
s_and_p_data = s_and_p_data.sort_values(by=['title'])
merged_fortune_data = pd.merge_asof(original_fortune_data, s_and_p_data, on = 'title' , direction='nearest')

TypeError: No matching signature found